This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'yv-sRAZRVJ49g_RmEHBW'

In [31]:
from platform import python_version

print(python_version())

3.7.6


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [70]:
# First, import the relevant modules

import json, requests

In [54]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

database_code = 'FSE'  # Frankfurt Stock Exchange
dataset_code = 'AFX_X'  # Carl Zeiss Meditec
start_date = '2017-01-05'
end_date = '2017-01-05'
url = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'

print(url)

response = requests.get(url)

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-05&end_date=2017-01-05&api_key=yv-sRAZRVJ49g_RmEHBW


In [55]:
response

<Response [200]>

In [56]:
json_response = json.loads(response.content)

In [57]:
json_response

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-01-05',
  'end_date': '2017-01-05',
  'frequency': 'daily',
  'data': [['2017-01-05',
    35.02,
    35.2,
    34.73,
    35.06,
    None,
    48412.0,
    1692326.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [58]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_response['dataset_data']['data'][0]

['2017-01-05',
 35.02,
 35.2,
 34.73,
 35.06,
 None,
 48412.0,
 1692326.0,
 None,
 None,
 None]

In [59]:
json_response['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [60]:
# 1. Collect data
database_code = 'FSE'
dataset_code = 'AFX_X'
start_date = '2017-01-01'
end_date = '2017-12-31'
order = 'asc'
url = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?start_date={start_date}&end_date={end_date}&order={order}&api_key={API_KEY}'

print(url)

response = requests.get(url)

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&order=asc&api_key=yv-sRAZRVJ49g_RmEHBW


In [61]:
response

<Response [200]>

In [62]:
# 2. Convert the returned JSON object into a Python dictionary
json_response = json.loads(response.content)

In [63]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
field_index = {j: i for i, j in enumerate(json_response['dataset_data']['column_names'])}
field_index

{'Date': 0,
 'Open': 1,
 'High': 2,
 'Low': 3,
 'Close': 4,
 'Change': 5,
 'Traded Volume': 6,
 'Turnover': 7,
 'Last Price of the Day': 8,
 'Daily Traded Units': 9,
 'Daily Turnover': 10}

In [64]:
opens = [day_data[field_index['Open']] for day_data in json_response['dataset_data']['data'] if day_data[field_index['Open']] is not None]

In [65]:
max(opens)

53.11

In [73]:
min(opens)

34.0

In [67]:
# 4. What was the largest change in any one day (based on High and Low price)?

day_changes = [round(day_data[field_index['High']] - day_data[field_index['Low']], 2) for day_data in json_response['dataset_data']['data'] if day_data[field_index['Low']] is not None and day_data[field_index['High']] is not None]

In [74]:
# this is an absolute value: we can't know based on the information provided if the stock went from high to low or low to high during the day
max(day_changes)

2.81

In [87]:
# 5. What was the largest change between any two days (based on Closing Price)?
# we use days that are consecutive in the dataset even if they are not absolutely consecutive
# also should come up with the change that is greatest in absolute value

data = json_response['dataset_data']['data']

max_close_change = 0.00

for day_1_index in range(len(data) - 1):
    day_2_index = day_1_index + 1
    day_1_close = data[day_1_index][field_index['Close']]
    day_2_close = data[day_2_index][field_index['Close']]
    if day_1_close is not None and day_2_close is not None:
        close_change = round(day_2_close - day_1_close, 2)
        if abs(max_close_change) < abs(close_change):
            max_close_change = close_change

print(max_close_change)

-2.56


In [88]:
# 6. What was the average daily trading volume during this year?

volumes = [day_data[field_index['Traded Volume']] for day_data in json_response['dataset_data']['data'] if day_data[field_index['Traded Volume']] is not None]
print(sum(volumes) / len(volumes))


89124.33725490196


In [91]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

def get_median(input_list):
    my_list = sorted(input_list)
    n = len(my_list)
    return (my_list[n // 2] + my_list[n // 2 - 1]) / 2 if n % 2 == 0 else my_list[n // 2]

In [92]:
get_median(volumes)

76286.0